In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        '''print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import librosa 
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
import librosa.display

In [ ]:
df=pd.read_csv('../input/urbansound8k/UrbanSound8K.csv')
df.head()

In [ ]:
df1=df[df['fold']==1]
df1['audio_load']=df1['slice_file_name'].apply(lambda x: librosa.load('../input/urbansound8k/fold1/'+x))

In [ ]:
plt.figure(figsize=(20, 5))
librosa.display.waveplot(df1['audio_load'].iloc[0][0], sr=df1['audio_load'].iloc[0][1])

In [ ]:

X = librosa.stft(df1['audio_load'].iloc[0][0])
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(20, 5))
librosa.display.specshow(Xdb, sr=df1['audio_load'].iloc[0][1], x_axis='time', y_axis='hz')
plt.colorbar()

In [ ]:
s
print(df1['audio_load'].iloc[0][0].shape)
#sampling rate 22050 default
'''23 ms window for fft algorithm because n_fft when set to power of 2 leads to faster calculation 
and hop_length recommended value = n_fft/4''' 
mfccs = librosa.feature.mfcc(df1['audio_load'].iloc[0][0], sr=df1['audio_load'].iloc[0][1],n_fft=512,hop_length=128) 


In [ ]:
sampling_rate=22050

In [ ]:
plt.figure(figsize=(10,4))
librosa.display.specshow(mfccs, x_axis="time")
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()
plt.show()

In [ ]:
df2=df.copy()

In [ ]:
#loading all audio files at once
df2['audio_load']=df2[['slice_file_name','fold']].apply(lambda x: librosa.load('../input/urbansound8k/fold{}/'.format(x['fold'])+x['slice_file_name']),axis=1)

In [ ]:
df2.to_csv('urbanaudio.csv')

In [ ]:
df2=pd.read_csv('./urbanaudio.csv')

In [ ]:
type(df2['audio_load'][0])

In [ ]:
def converter(instr):
    return np.fromstring(instr[1:-1],sep=' ')
df2=pd.read_csv('./urbanaudio.csv',converters={'audio_load':converter})

In [ ]:
df2['audio_load'][0]

In [ ]:
MAX_LENGTH=690 # max length of mfcc features since audio<=4 seconds
df2['mfcc']=df2['audio_load'].apply(lambda x: librosa.feature.mfcc(x[0],sr=sampling_rate,n_fft=512,hop_length=128))
df2['mfcc']=df2['mfcc'].apply(lambda x: tf.keras.preprocessing.sequence.pad_sequences(x, padding="post",maxlen=690,dtype='float32'))

In [ ]:
import pickle
with open('array.pickle', 'wb') as f:
    data=df2['audio_load'].to_list()
    pickle.dump(data, f)

In [ ]:
def process(dataframe):
    mfcc_list=[]
    mfcc_list+=dataframe['mfcc'].to_list()
    X=np.array(mfcc_list)
    X=X.swapaxes(1,2) # for lstm input shape
    y=dataframe['classID'].values
    return X,y

**Using MFCC as feature**

LSTM model

In [ ]:
results=[] # save accuracy for each fold
for i in range(10):
    print('\nFold{} as validation set model training\n'.format(i+1))
    df_val=df2[df2['fold']==i+1]
    df_train=df2[df2['fold']!=(i+1)]
    train_x,train_y=process(df_train)
    val_x,val_y=process(df_val)
    #690 max time steps and 20 mfcc coefficients
    mfcc_feature = tf.keras.Input(shape=(690,20), name="mfcc_feature") # variable length with padding and mask 
    mask_layer=tf.keras.layers.Masking(mask_value=0.,input_shape=(690,20))(mfcc_feature)
    #conv_layer=tf.keras.layers.Conv1D(32, 3, activation='relu',input_shape=(690,20))(mask_layer)
    #flat=tf.keras.layers.Flatten()(conv_layer)
    state_h=tf.keras.layers.LSTM(256,name='lstm')(mask_layer)
    hidden_1=tf.keras.layers.Dense(128,activation='relu',name='hidden1')(state_h)
    drop_1=tf.keras.layers.Dropout(0.5)(hidden_1)
    hidden_2=tf.keras.layers.Dense(64,activation='relu',name='hidden2')(drop_1)
    drop_2=tf.keras.layers.Dropout(0.5)(hidden_2)
    pred=tf.keras.layers.Dense(10,name='prediction')(drop_2)
    model=tf.keras.Model(inputs=mfcc_feature,outputs=pred,name='fold{}_val'.format(i+1))
    tf.keras.utils.plot_model(model, "lstm_model.png", show_shapes=True)
    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam'
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    model.fit(train_x,train_y,validation_data=(val_x, val_y),epochs=10,batch_size=128)
    model.save('fold{}_val_model'.format(i+1))
    results.append(model.evaluate(val_x, val_y)[1])
    

In [ ]:
print('10-fold cross validation accuracy using LSTM model: ',(np.array(results)).mean())

GRU model

In [ ]:
results=[] # save accuracy for each fold
for i in range(10):
    print('\nFold{} as validation set model training\n'.format(i+1))
    df_val=df2[df2['fold']==i+1]
    df_train=df2[df2['fold']!=(i+1)]
    train_x,train_y=process(df_train)
    val_x,val_y=process(df_val)
    #690 max time steps and 20 mfcc coefficients
    mfcc_feature = tf.keras.Input(shape=(690,20), name="mfcc_feature") # variable length with padding and mask 
    mask_layer=tf.keras.layers.Masking(mask_value=0.,input_shape=(690,20))(mfcc_feature)
    #conv_layer=tf.keras.layers.Conv1D(32, 3, activation='relu',input_shape=(690,20))(mask_layer)
    #flat=tf.keras.layers.Flatten()(conv_layer)
    state_h=tf.keras.layers.GRU(128,name='GRU')(mask_layer)
    hidden_1=tf.keras.layers.Dense(128,activation='relu',name='hidden1')(state_h)
    drop_1=tf.keras.layers.Dropout(0.5)(hidden_1)
    hidden_2=tf.keras.layers.Dense(64,activation='relu',name='hidden2')(drop_1)
    drop_2=tf.keras.layers.Dropout(0.5)(hidden_2)
    pred=tf.keras.layers.Dense(10,name='prediction')(drop_2)
    model=tf.keras.Model(inputs=mfcc_feature,outputs=pred,name='fold{}_val'.format(i+1))
    tf.keras.utils.plot_model(model, "lstm_model.png", show_shapes=True)
    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam'
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    model.fit(train_x,train_y,validation_data=(val_x, val_y),epochs=15,batch_size=128)
    model.save('fold{}_val_model'.format(i+1))
    results.append(model.evaluate(val_x, val_y)[1])
    

Models overfits more in GRU case

In [ ]:
print('10-fold cross validation accuracy using GRU model: ',(np.array(results)).mean())

LSTM on normalized dataset

In [ ]:
from sklearn.preprocessing import normalize
#results=[] # save accuracy for each fold
#for i in range(10):
i=0
print('\nFold{} as validation set model training\n'.format(i+1))
df_val=df2[df2['fold']==i+1]
df_train=df2[df2['fold']!=(i+1)]
train_x,train_y=process(df_train)
val_x,val_y=process(df_val)
# normalize along timesteps
for i,a in enumerate(train_x):
    train_x[i]=normalize(a,axis=0)
for i,a in enumerate(val_x):
    val_x[i]=normalize(a,axis=0)

#690 max time steps and 20 mfcc coefficients
mfcc_feature = tf.keras.Input(shape=(690,20), name="mfcc_feature") # variable length with padding and mask 
mask_layer=tf.keras.layers.Masking(mask_value=0.,input_shape=(690,20))(mfcc_feature)
#conv_layer=tf.keras.layers.Conv1D(32, 3, activation='relu',input_shape=(690,20))(mask_layer)
#flat=tf.keras.layers.Flatten()(conv_layer)
state_h=tf.keras.layers.LSTM(128,name='lstm')(mask_layer)
hidden_1=tf.keras.layers.Dense(128,activation='relu',name='hidden1')(state_h)
drop_1=tf.keras.layers.Dropout(0.5)(hidden_1)
hidden_2=tf.keras.layers.Dense(64,activation='relu',name='hidden2')(drop_1)
drop_2=tf.keras.layers.Dropout(0.5)(hidden_2)
pred=tf.keras.layers.Dense(10,name='prediction')(drop_2)
model=tf.keras.Model(inputs=mfcc_feature,outputs=pred,name='fold{}_val'.format(i+1))
tf.keras.utils.plot_model(model, "lstm_model.png", show_shapes=True)
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
    ),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(train_x,train_y,validation_data=(val_x, val_y),epochs=30,batch_size=128)
#model.save('fold{}_val_model'.format(i+1))
#results.append(model.evaluate(val_x, val_y)[1])


With normalized data we observe drop in accuracy even with more epochs

Also using mfcc delta and double delta features without normalizing

In [ ]:
df2['mfcc_delta']=df2['mfcc'].apply(lambda x: librosa.feature.delta(x))
df2['mfcc_double_delta']=df2['mfcc'].apply(lambda x: librosa.feature.delta(x,order=2))

In [ ]:
def process2(dataframe):
    mfcc_list=[]
    mfcc_delta_list=[]
    mfcc_double_delta_list=[]
    
    mfcc_list+=dataframe['mfcc'].to_list()
    X1=np.array(mfcc_list)
    X1=X1.swapaxes(1,2) # for lstm input shape
    
    mfcc_delta_list+=dataframe['mfcc_delta'].to_list()
    X2=np.array(mfcc_delta_list)
    X2=X2.swapaxes(1,2) # for lstm input shape
    
    mfcc_double_delta_list+=dataframe['mfcc_double_delta'].to_list()
    X3=np.array(mfcc_double_delta_list)
    X3=X3.swapaxes(1,2) # for lstm input shape
    
    y=dataframe['classID'].values
    return X1,X2,X3,y

In [ ]:
results=[] # save accuracy for each fold
for i in range(10):
    print('\nFold{} as validation set model training\n'.format(i+1))
    df_val=df2[df2['fold']==i+1]
    df_train=df2[df2['fold']!=(i+1)]
    m_x,md_x,mdd_x,train_y=process2(df_train)
    val_m_x,val_md_x,val_mdd_x,val_y=process2(df_val)
    #690 max time steps and 20 mfcc coefficients
    mfcc = tf.keras.Input(shape=(690,20), name="mfcc") # variable length with padding and mask 
    mfcc_delta = tf.keras.Input(shape=(690,20), name="mfcc_delta") # variable length with padding and mask 
    mfcc_double_delta = tf.keras.Input(shape=(690,20), name="mfcc_double_delta") # variable length with padding and mask 
    mask_layer1=tf.keras.layers.Masking(mask_value=0.,input_shape=(690,20))(mfcc)
    mask_layer2=tf.keras.layers.Masking(mask_value=0.,input_shape=(690,20))(mfcc_delta)
    mask_layer3=tf.keras.layers.Masking(mask_value=0.,input_shape=(690,20))(mfcc_double_delta)
    #conv_layer=tf.keras.layers.Conv1D(32, 3, activation='relu',input_shape=(690,20))(mask_layer)
    #flat=tf.keras.layers.Flatten()(conv_layer)
    state_h1=tf.keras.layers.LSTM(64,name='lstm1')(mask_layer1)
    state_h2=tf.keras.layers.LSTM(64,name='lstm2')(mask_layer2)
    state_h3=tf.keras.layers.LSTM(64,name='lstm3')(mask_layer3)
    features= tf.keras.layers.concatenate([state_h1, state_h2, state_h3])
    hidden_1=tf.keras.layers.Dense(128,activation='relu',name='hidden1')(features)
    drop_1=tf.keras.layers.Dropout(0.5)(hidden_1)
    hidden_2=tf.keras.layers.Dense(64,activation='relu',name='hidden2')(drop_1)
    drop_2=tf.keras.layers.Dropout(0.5)(hidden_2)
    pred=tf.keras.layers.Dense(10,name='prediction')(drop_2)
    model=tf.keras.Model(inputs=[mfcc,mfcc_delta,mfcc_double_delta],outputs=pred,name='fold{}_val'.format(i+1))
    tf.keras.utils.plot_model(model, "lstm_model.png", show_shapes=True)
    model.compile(optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
        name='Adam'
        ),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    model.fit({"mfcc": m_x, "mfcc_delta": md_x, "mfcc_double_delta":mdd_x },train_y,validation_data=({"mfcc": val_m_x, "mfcc_delta": val_md_x, "mfcc_double_delta":val_mdd_x }, val_y),epochs=20,batch_size=128)
    model.save('fold{}_val_model'.format(i+1))
    results.append(model.evaluate(val_x, val_y)[1])
